In [1]:
import pandas as pd 
import numpy as np
import nltk
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
import re
import collections
import operator
import os
import pickle
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import doc2vec
from collections import namedtuple



data preprocessing:

When reading the file, will skip the first 3000 characters, which are same text for all the 10k files.
In order to reduce the size of words and eliminate the effects of redundant words, we used the stopwords package from nltk to exclude some common stopwors. However, the nltk package only contains limited amount of stopwords, which was not enough for our data pre-processing. We count the frequency of random selected files in 1996 & 2004 and chose 50 most frequent words for each 10k files and manually select words to include those words in our stop word list.

In [91]:
f = open('/Users/jinbinliang/Desktop/ISE599/Project/10K/1996.full/06652B103-10-K-19961216.txt','r')

In [195]:
# Read the whole file
with open('/Users/jinbinliang/Desktop/ISE599/Project/10K/1996.full/06652B103-10-K-19961216.txt','r') as f:
    
    pre_processed_article = f.read()[3000:]

f.close()


# Data-preprocesssing, exclude special character / space / number 

In [2]:
# Pre-process file, set exclude_digit = True if do not include digit 
def pre_process(file_content, exclude_digit = False):
    
    processed_article = file_content.lower()
    
    # Decide whether to exclude the digit or not 
    if exclude_digit is False:
    
        # Include digit
        processed_article = re.sub(',', '', processed_article )
        processed_article = re.sub('[^a-zA-Z0-9]', ' ', processed_article )
    else: 
        # Exclude digit
        processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )

    processed_article = re.sub(r'\s+', ' ', processed_article)
    
    
    return processed_article

# Get more stop words from the 10k content through observing the word frequency

In [4]:
def get_count_of_word_after_remove_stopwords(processed_article):
    all_sentences = nltk.sent_tokenize(processed_article)
    
    if len(all_sentences) == 1:
        all_words = nltk.word_tokenize(all_sentences[0])
        print('There are',len(all_words),'words.')
        
        all_words = [word for word in all_words if len(word) >4 ]
        print('There are',len(all_words),'has more than 4 characters.')

        
        all_words = [word for word in all_words if word not in stopwords.words('english')]
        print('There are',len(all_words),'after removed stop_words.\n')
        
        Freq_word=collections.Counter(all_words)
        sorted_x = sorted(Freq_word.items(), key=operator.itemgetter(1), reverse = True)
        
        Most_Freq = []
        for n in range(0,50):
            Most_Freq.append(sorted_x[n][0])
        
        return Most_Freq
    else:
        print('Length > 1.')
        
            

# Add new stop_words to the stop words list

In [5]:
# Add more words to the stop_words list
file_name_1996 = os.listdir('/Users/jinbinliang/Desktop/ISE599/Project/10K/1996.full')

for i in range(1000,1020):
    file_name = file_name_1996[i]
    file_path = '/Users/jinbinliang/Desktop/ISE599/Project/10K/1996.full/'+file_name
    f = open(file_path,'r')
    pre_processed_article = f.read()[3000:]
    f.close()
    processed_article = pre_process(pre_processed_article)
    list_word = get_count_of_word_after_remove_stopwords(processed_article)
    print(file_name,'\n')
    print(list_word,'\n')

There are 117115 words.
There are 45382 has more than 4 characters.
There are 42949 after removed stop_words.

542312103-10-K-19960328.txt 

['tract', 'corner', 'survey', 'thence', 'distance', 'steel', 'total', 'seller', 'texas', 'shall', 'along', 'north', 'south', 'acres', 'oxygen', 'county', 'described', 'property', 'agreement', 'stock', 'stake', 'exhibit', 'company', 'beginning', 'buyer', 'northwest', 'southwest', 'northeast', 'southeast', 'dated', 'preferred', 'purchaser', 'follows', 'number', 'plant', 'morris', 'right', 'boundary', 'marked', 'million', 'article', 'monument', 'shares', 'contract', 'containing', 'volume', 'records', 'except', 'corporation', 'place'] 

There are 17842 words.
There are 8043 has more than 4 characters.
There are 7657 after removed stop_words.

023559206-10-K-19961127.txt 

['company', 'stock', 'shares', 'lease', 'financial', 'series', 'common', 'preferred', 'statements', 'interest', 'payable', 'sodium', 'investment', 'notes', 'trust', 'agreement', 'sch

There are 26956 words.
There are 12844 has more than 4 characters.
There are 12234 after removed stop_words.

45811K109-10-K-19960926.txt 

['company', 'employee', 'shall', 'agreement', 'fiscal', 'stock', 'million', 'financial', 'income', 'products', 'employment', 'common', 'turtle', 'beach', 'revenue', 'section', 'provided', 'statements', 'statement', 'product', 'consolidated', 'years', 'certain', 'business', 'effective', 'sales', 'table', 'development', 'options', 'agreements', 'accounting', 'provisions', 'shares', 'including', 'assets', 'period', 'registrant', 'market', 'prior', 'purchase', 'pursuant', 'quarter', 'total', 'respect', 'share', 'ended', 'termination', 'value', 'interest', 'expenses'] 

There are 85550 words.
There are 40671 has more than 4 characters.
There are 38946 after removed stop_words.

148449309-10-K-19960315.txt 

['company', 'castle', 'shall', 'agreement', 'section', 'september', 'party', 'corporation', 'refining', 'buyer', 'assets', 'indian', 'closing', 'sto

# Use this add_stop_words

In [9]:
add_stop_word = ['0','1','2','3','4','5','6','7','8','9','10','a','b','c','e','f','g','h','i','j','k','l','m','n','o',
                 'p','q','r','s','t','u','v','w','s','y','z','ltd','part','000','note','year','site','ended',
                 'company','section','i','ii','shall','may','report','date','item','form','time','page','inc',
                'table','article','line','10k','file','person','shall','first','three','years','products']

In [7]:
New_stop_words = add_stop_word + stopwords.words('english')
len(New_stop_words)

245

In [12]:
# write stop_words to file
with open("/Users/jinbinliang/Desktop/ISE599/Project/10K/stop_word_new.txt", "w") as f:
    for word in New_stop_words:
        f.write(word +"\n")
        
# # read stop_words to file
# with open("/Users/jinbinliang/Desktop/ISE599/Project/10K/stop_word.txt", "r") as f:
#       for line in f:
#     score.append(int(line.strip()))

# Using new stop words list to reduce size of content

In [25]:
def Exclude_Stop_words (processed_article, stop_words_list):
    all_sentences = nltk.sent_tokenize(processed_article)
    
    if len(all_sentences) == 1:
        all_words = nltk.word_tokenize(all_sentences[0])
        print('There are',len(all_words),'words.')
        
        all_words = [word for word in all_words if len(word) >4 ]
        print('There are',len(all_words),'has more than 4 characters.')

        all_words = [word for word in all_words if word not in stop_words_list]
        print('There are',len(all_words),'after removed stop_words.\n')
        
        return all_words
    
    else:
        print('Length > 1.')

# Build word2vec model

In [8]:
# Remove words by new stop_words_list
file_name_1996 = os.listdir('/Users/jinbinliang/Desktop/ISE599/Project/10K/2004.full')
stop_words_list = []
list_10k_contents = []
with open("/Users/jinbinliang/Desktop/ISE599/Project/10K/stop_word.txt", "r") as f:
    for line in f:
        stop_words_list.append(str(line.strip()))


for i in range(1000,1020):
    file_name = file_name_1996[i]
    file_path = '/Users/jinbinliang/Desktop/ISE599/Project/10K/2004.full/'+file_name
    f = open(file_path,'r')
    pre_processed_article = f.read()[3000:]
    f.close()
    processed_article = pre_process(pre_processed_article)
    Exclude_word_processed_article = Exclude_Stop_words(processed_article,stop_words_list)
    list_10k_contents.append(Exclude_word_processed_article)

In [47]:
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(list_10k_contents):
    tags = [i]
    docs.append(analyzedDocument(text, tags))

In [50]:
model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 2, workers = 4)

/Users/jinbinliang/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
